In [69]:
%run "../config.py"

declared 4 variables


In [90]:
from keras import models
import json

import keras
from keras import Sequential, layers, activations, optimizers
from keras.layers import Conv2D, Dense, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3

import os

In [70]:
with open(MODELS_DIR.joinpath("model1.json"), "r") as f:
    config = f.read()

In [95]:
train_dir = os.path.join(TMP_DIR.joinpath("sign-language-img.zip"), "Train")
test_dir = os.path.join(TMP_DIR.joinpath("sign-language-img.zip"), "Test")

train_datagen = ImageDataGenerator(
    rescale = 1/255
)
test_datagen = ImageDataGenerator(
    rescale = 1/255,
    validation_split = 0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = "categorical",
    color_mode = "rgb",
    subset = "training"
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = "categorical",
    color_mode = "rgb",
    subset = "validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = "categorical",
    color_mode = "rgb",

)

Found 27455 images belonging to 24 classes.
Found 1425 images belonging to 24 classes.
Found 7172 images belonging to 24 classes.


In [78]:
pre_trained_model = InceptionV3(
    include_top = False,
    input_tensor = Input(shape = (150,150, 3))
)

for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
model = models.load_model(str(MODELS_DIR.joinpath("model1.hdf5")))

INFO:plaidml:Opening device "opencl_amd_gfx1010:xnack-.0"


In [81]:
pre_trained_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, 74, 74, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, 74, 74, 32)   96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_283 (Activation)     (None, 74, 74, 32)   0           batch_normalization_283[0][0]    
__________________________________________________________________________________________________
conv2d_284

In [82]:
last_layer = pre_trained_model.get_layer("mixed7")
last_output = last_layer.output

In [86]:
import pickle

In [87]:
with open(DATA_DIR.joinpath("decoder.pickle"), "rb") as f:
    classes = pickle.load(f)

In [92]:
x = layers.Flatten()(last_output)
x = layers.Dense(128, activation=activations.relu)(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(len(classes), activation=activations.softmax)(x)

model_keras = keras.Model(pre_trained_model.input, x)
model_keras.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"]
)

In [93]:
model_keras.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, 74, 74, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, 74, 74, 32)   96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_283 (Activation)     (None, 74, 74, 32)   0           batch_normalization_283[0][0]    
__________________________________________________________________________________________________
conv2d_284

In [96]:
history_keras = model_keras.fit_generator(
    train_generator,
    epochs = 5,
    validation_data= validation_generator
)

Epoch 1/5
682/858 [======================>.......] - ETA: 26s - loss: 0.1116 - categorical_accuracy: 0.9680

INFO:plaidml:Analyzing Ops: 913 of 1561 operations complete


858/858 [==============================] - 167s 195ms/step - loss: 0.1004 - categorical_accuracy: 0.9715 - val_loss: 1.9239 - val_categorical_accuracy: 0.7347
Epoch 2/5
858/858 [==============================] - 79s 92ms/step - loss: 0.0271 - categorical_accuracy: 0.9919 - val_loss: 2.9455 - val_categorical_accuracy: 0.6667
Epoch 3/5
858/858 [==============================] - 77s 90ms/step - loss: 0.0288 - categorical_accuracy: 0.9923 - val_loss: 2.3632 - val_categorical_accuracy: 0.7018
Epoch 4/5
858/858 [==============================] - 77s 90ms/step - loss: 0.0357 - categorical_accuracy: 0.9913 - val_loss: 2.3954 - val_categorical_accuracy: 0.7158
Epoch 5/5
858/858 [==============================] - 77s 90ms/step - loss: 0.0234 - categorical_accuracy: 0.9945 - val_loss: 2.1953 - val_categorical_accuracy: 0.7740


In [97]:
model_keras.evaluate_generator(test_generator)

[2.241576824916918, 0.7625488008923592]